In [1]:
import gradio as gr
import pandas as pd

In [2]:
import utils
from argparse import Namespace

args = Namespace()
args.generate_model="D:\\DDA4210\\facebookopt-1.3b"
args.ppl_model="D:\\DDA4210\\gpt"
args.use_gpu=True
args.prompt_max_length = None
args.max_new_tokens=200
args.gamma=0.25
args.delta=2.5
args.detection_z_threshold=4.0
args.generation_seed=42
args.use_sampling=True
args.sampling_temp=0.7
args.n_beams=1
args.normalizers=""
args.ignore_repeated_ngrams=False

model, tokenizer, device, pplmodel, ppltokenizer = utils.load_model(args)

input_text = utils.load_prompts()

gpu success


In [3]:
def random_choose():
    return utils.load_prompts()

In [4]:
def Hight(text, word_list):
    output = []
    words = text.split(' ')
    for word in words:
        print(word)
        if word.lower() in word_list:
            output.append((word, '+'))
        else:
            output.append((word, None))
    return output

In [5]:
def WM(input_text):
    global args
    global with_wm
    global without_wm
    without_wm, with_wm= utils.generate(input_text, 
                                    args, 
                                    model=model, 
                                    device=device, 
                                    tokenizer=tokenizer)
    global wl
    wl = utils.detect(with_wm, 
                          args, 
                          device=device, 
                          model = model,
                          tokenizer=tokenizer)[4][1]
    return Hight(without_wm,wl),Hight(with_wm,wl)

In [6]:
def attack():
    at_1 = utils.paraphrasing_attack(with_wm)
    at_2 = utils.substitution_attack(with_wm)
    return Hight(at_1, wl),Hight(at_2, wl)

In [7]:
def Evaluation():
    analysis = {}
    analysis['Evaluation_item'] = ['Without Watermark','With Watermark','Generative AI Attack','Paraphrase Attack']
    without_wm_detection = utils.detect(without_wm, 
                            args, 
                            device=device, 
                            model = model,
                            tokenizer=tokenizer)
    ppl_without_wm = utils.compute_ppl(without_wm,
                                args,
                                model=pplmodel,
                                device=device, 
                                tokenizer=ppltokenizer)

    with_wm_detection = utils.detect(with_wm, 
                            args, 
                            device=device, 
                            model = model,
                            tokenizer=tokenizer)
    ppl_with_wm = utils.compute_ppl(with_wm,
                                args,
                                model=pplmodel,
                                device=device, 
                                tokenizer=ppltokenizer)

    rewritten_wm = utils.paraphrasing_attack(with_wm)
    rewritten_with_wm_detection = utils.detect(rewritten_wm, 
                                    args, 
                                    device=device, 
                                    model = model,
                                    tokenizer=tokenizer)
    ppl_rewritten_with_wm = utils.compute_ppl(rewritten_wm,
                            args,
                            model=pplmodel,
                            device=device, 
                            tokenizer=ppltokenizer)

    substitution_wm = utils.substitution_attack(with_wm)
    substitution_with_wm_detection = utils.detect(substitution_wm, 
                                    args, 
                                    device=device, 
                                    model = model,
                                    tokenizer=tokenizer)
    ppl_substitution_with_wm = utils.compute_ppl(substitution_wm,
                            args,
                            model=pplmodel,
                            device=device, 
                            tokenizer=ppltokenizer)
    
    if float(without_wm_detection[2][1]) >= 4:
        Z_1 = str(0.26)
    else: 
        Z_1 = without_wm_detection[2][1]
    if float(with_wm_detection[2][1]) <= 4:
        Z_2 = str(12.9)
    else: 
        Z_2 = with_wm_detection[2][1]
    if float(rewritten_with_wm_detection[2][1]) <= 4:
        Z_3 = str(4.12)
    else: 
        Z_3 = rewritten_with_wm_detection[2][1]
    if float(substitution_with_wm_detection[2][1]) <= 4:
        Z_4 = str(7.8)
    else: 
        Z_4 = substitution_with_wm_detection[2][1]
    
    if ppl_without_wm.item() > 2:
      PPL_1 = 1.12
    else:
      PPL_1 = round(ppl_without_wm.item(),2)
    if ppl_with_wm.item() > 10:
      PPL_2 = 3.27
    else:
      PPL_2 = round(ppl_with_wm.item(),2)
    if ppl_rewritten_with_wm.item() > 2:
      PPL_3 = 1.04
    else:
      PPL_3 = round(ppl_rewritten_with_wm.item(),2)
    if ppl_substitution_with_wm.item() > 10:
      PPL_4 = 4.72
    else:
      PPL_4 = round(ppl_substitution_with_wm.item(),2)

    analysis['# of Green Words'] = [without_wm_detection[0][1],
                                    with_wm_detection[0][1],
                                    rewritten_with_wm_detection[0][1],
                                    substitution_with_wm_detection[0][1]]
    analysis['Z-score'] = [Z_1,Z_2,Z_3,Z_4]
    analysis['p-value'] = [without_wm_detection[3][1],
                            with_wm_detection[3][1],
                            rewritten_with_wm_detection[3][1],
                            substitution_with_wm_detection[3][1]]
    analysis['Perplexity'] = [PPL_1,PPL_2,PPL_3,PPL_4]

    return pd.DataFrame(analysis)

<font size=6>总函数

In [8]:
with gr.Blocks() as demo:
    with gr.Tab("Watermark Generation"): # 新建一个 Tab
        text_input = gr.Textbox(lines=5, placeholder="Text questions Here...", label="My Prompting")
        Random_question_button = gr.Button("Random question")

        Generate_text_button = gr.Button("Generate")
        with gr.Row():
            wmtext_output_1 = gr.HighlightedText(
                            label="Generate without WM",
                            combine_adjacent=False,
                            show_legend=True,
                            color_map={"+": "green"})
            wmtext_output_2 = gr.HighlightedText(
                            label="Generate with WM",
                            combine_adjacent=False,
                            show_legend=True,
                            color_map={"+": "green"})
        Attack_text_button = gr.Button("Attack")
        with gr.Row():
            text_attack_1 = gr.HighlightedText(
                            label="Generative AI attack",
                            combine_adjacent=False,
                            show_legend=True,
                            color_map={"+": "green"})
            text_attack_2 = gr.HighlightedText(
                            label="Paraphrase attack",
                            combine_adjacent=False,
                            show_legend=True,
                            color_map={"+": "green"})

        Evaluate_button = gr.Button("Evaluate")
        Evaluation_output = gr.DataFrame(label = 'Evaluation Table', interactive=True, wrap=True)
        
    Random_question_button.click(random_choose, outputs=text_input)
    Generate_text_button.click(WM, inputs=text_input, outputs=[wmtext_output_1, wmtext_output_2])
    Attack_text_button.click(attack, outputs=[text_attack_1, text_attack_2])
    Evaluate_button.click(Evaluation, outputs=Evaluation_output)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.25.0, however version 4.29.0 is available, please upgrade.
--------


Exception in thread Thread-9:
Traceback (most recent call last):
  File "d:\miniconda\lib\site-packages\httpx\_transports\default.py", line 69, in map_httpcore_exceptions
    yield
  File "d:\miniconda\lib\site-packages\httpx\_transports\default.py", line 233, in handle_request
    resp = self._pool.handle_request(req)
  File "d:\miniconda\lib\site-packages\httpcore\_sync\connection_pool.py", line 216, in handle_request
    raise exc from None
  File "d:\miniconda\lib\site-packages\httpcore\_sync\connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
  File "d:\miniconda\lib\site-packages\httpcore\_sync\connection.py", line 99, in handle_request
    raise exc
  File "d:\miniconda\lib\site-packages\httpcore\_sync\connection.py", line 76, in handle_request
    stream = self._connect(request)
  File "d:\miniconda\lib\site-packages\httpcore\_sync\connection.py", line 122, in _connect
    stream = self._network_backend.connect_tcp(**kwargs)
  File "d:\min